In [1]:
%pylab inline
import pandas as pd
import os
from textwrap import dedent

Populating the interactive namespace from numpy and matplotlib


In [2]:
assembly_common_name_map = {'mm10': 'mouse', 'hg38': 'human'}

sheet_name = 'hg38'
excel_df = pd.read_excel('../data/datasets_info.xlsx', sheet_name=sheet_name)


In [3]:
excel_df

,GEO,SRA,RIBO sample,RNA sample,read lengths,offsets,RIBO adapter,RNA adapter,RIBO strandedness,RNA strandedness,AUC(RiboCop/RiboCode/RiboTaper/ORFscore),metagene_coherence_ribocop
0,GSE35469,SRP010679,SRX118286,SRX118285,"28,29,30","12,13,13",CTGTAGGCAC,CTGTAGGCAC,forward,forward,NaN,NaN
1,NaN,SRP029589,SRX345309,SRX345311,"29,30,32","12,12,13",CTGTAGGCACCATCAAT,CTGTAGGCACCATCAAT,forward,forward,NaN,NaN
2,GSE73136,SRP063852,SRX1254413,"SRX426378 (SRP028887, GSE49831)","28,29,30","12,12,12",None,None,forward,forward,NaN,NaN
3,GSE94454,SRP098789,SRX2536421,SRX2536426,"28,30","12,13",CTGTAGGCACCATCAAT,CTGTAGGCACCATCAAT,forward,forward,NaN,NaN
4,GSE96716,SRP102021,SRX2647167,SRX2647164,"28,29,30,31","12,12,12,12",TCGTATGCCGTCTTCTGCTTG,None,forward,forward,NaN,NaN
5,GSE87328,SRP090415,SRX2189157,SRX2189154,"26,27,28,29,30","12,12,12,12,12",CTGTAGGCACCATCAAT,AGATCGGAAGAGCACAC,forward,reverse,NaN,NaN
6,GSE89183,SRP092068,SRX2268404,SRX2268396,"26,27,28,29,30","12,12,12,12,12",AGATCGGAAGAGCACACGTCT,None,forward,reverse,NaN,NaN
7,GSE96714,SRP102020,SRX2647162,SRX2647160,"29,30","12,12",TCGTATGCCGTCTTCTGCTTG,None,forward,no,NaN,NaN


In [4]:
output_dir = '/home/cmb-panasas2/skchoudh/github_projects/ribocop-results/tools/ribORF/offsets/hg38'

In [5]:
!mkdir -p /home/cmb-panasas2/skchoudh/github_projects/ribocop-results/tools/ribORF/offsets/hg38

In [6]:
offset_file_map = {}
for index, row in excel_df.iterrows():
    read_lengths = row['read lengths'].strip().split(',')
    offsets = row['offsets'].strip().split(',')
    srp = row['SRA']
    ribo_sample = row['RIBO sample']
    rna_sample = row['RIBO sample']
    ribo_outfile = os.path.join(output_dir, '{}_{}_{}.offset.txt'.format(srp, 'ribo', ribo_sample))
    rna_outfile = os.path.join(output_dir, '{}_{}_{}.offset.txt'.format(srp, 'rna', rna_sample))
    
    with open(rna_outfile, 'w') as fh:
        for read_length, offset in zip(read_lengths, offsets):
            fh.write('{}\t{}\n'.format(read_length, offset))
            
    with open(ribo_outfile, 'w') as fh:
        for read_length, offset in zip(read_lengths, offsets):
            fh.write('{}\t{}\n'.format(read_length, offset))
        
    offset_file_map['{}_human'.format(srp)] = ribo_outfile

In [8]:
input_dir_root = '/home/cmb-panasas2/wenzhenl/benchmark/gene_level/hg38/'
output_dir_root = '/staging/as/skchoudh/benchmark/gene_level/hg38/'
create_config_dir = '/home/cmb-panasas2/skchoudh/github_projects/ribocop-results/tools/ribORF/configs'
candidate_orfs = '/home/cmb-panasas2/skchoudh/genomes/hg38/ribORF_annotation/candidateORF.genepred.txt'
for srp in os.listdir(input_dir_root):
    print(srp)
    if srp=='test' or srp == 'SRP063852_human_old' or srp == 'positive_only_version':
        continue
    input_dir = os.path.join(input_dir_root, srp)
    output_dir = os.path.join(output_dir_root, srp)
    
    offset_file = offset_file_map[srp]
    config_file = os.path.join(create_config_dir, srp+'.py')
    config = dedent("""
    INPUT_DIR = '{}'
    OUT_DIR = '{}'
    OFFSET_FILE = '{}'
    CANDIDATE_ORFS = '{}'
    """).format(input_dir, output_dir, offset_file, candidate_orfs)
    with open(config_file, 'w') as fh:
        fh.write(config)
    

positive_only_version
SRP063852_human
SRP010679_human
SRP029589_human
SRP098789_human
SRP102021_human


In [ ]:
riboORF_src_dir = '/home/cmb-panasas2/skchoudh/github_projects/ribocop_tools_source/RibORF/RibORF.1.0/'
out_dir = os.path.join(out_dir, srp)


config = dedent('''
INPUT_DIR = {}
OFFSET_FILE = {}
''').format(input_dir, offset_file, out_dir)